In [ ]:
import time
from requests_html import HTMLSession
from lxml import html
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
USER_AGENT = os.getenv("USER_AGENT")

In [ ]:
class Scraper:
    def __init__(self, asin) -> None:
        self.session = HTMLSession()
        self.asin = asin
        self.url = f"https://www.amazon.com/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber="
    
    def check_page(self, i):
        headers = {"User-Agent": USER_AGENT}
        response = self.session.get(self.url + str(i), headers=headers)

        if response.html.find('div[data-hook=review]'):
            return response.html.find('div[data-hook=review]')
        else:
            return False

    def scrape(self, reviews):
        final = []

        for review in reviews:
            try:
                title = review.find('div[data-hook="review"] span[data-hook="review-title"]', first = True).text.strip().replace('\n', '')
                rating = review.find('i[data-hook=review-star-rating] span', first = True).text.strip().replace('\n', '').replace(' out of 5 stars', '')
                body = review.find('div[data-hook="review"] span[data-hook="review-body"]', first = True).text.strip().replace('\n', '')
            except:
                continue        

            data = {'title': title,
                    'rating': rating,
                    'body': body}      
            final.append(data)
        return final

In [ ]:
scraper = Scraper('B08D6VD9TR')

results = []
for i in range(1,51):          
    print('Getting page', i)
    time.sleep(0.3)             
    reviews = scraper.check_page(i)
    
    if reviews:                    
        results.append(scraper.scrape(reviews))
    else:
        print('No more reviews')


In [ ]:
results